In [ ]:
%cd ..

In [ ]:
import altair as alt
from altair_saver import save
import pandas as pd
from util import load_experiment

In [ ]:
baseline_df, val_df, test_df = load_experiment("feature_collisions")

In [ ]:
model2name = {
    "Neural PBM - Unbiased": "PBM - True Bias",
    "Neural PBM - Biased": "PBM - Naive",
    "Neural PBM - Estimated bias": "PBM - Estimated Bias",
    "Pointwise IPS - Unbiased": "Point. IPS - True Bias",
    "Pointwise IPS - Biased": "Point. IPS / PBM - Naive",
}

test_df.model = test_df.model.map(model2name)
test_df = test_df[(test_df.model != "PBM - Naive") & (test_df.model != "PBM - Estimated Bias")]

In [ ]:
def plot(dataset_df, legend=True, width=400, height=200, metric="average_relevant_position", title="", y=[0, 1.0], clip=False, label_y=True, label_x=True): 
    lines = alt.Chart(dataset_df, width=width, height=height, title=title).mark_line(clip=clip).encode(
        x=alt.X("data_perc_feature_collision", title="% of Feature Collisions", axis=alt.Axis(format='%')),
        y=alt.Y(f"mean({metric})", scale=alt.Scale(zero=False, domain=y), title=metric if label_y else None),
        color=alt.Color("model", legend=None),
        tooltip=[f"count({metric})", "n_sessions", f"mean({metric})"]
    )

    marks = alt.Chart(dataset_df).mark_point(clip=clip, size=50).encode(
        x=alt.X("data_perc_feature_collision"),
        y=alt.Y(f"mean({metric})", scale=alt.Scale(zero=False)),
        shape=alt.Shape("model"),
        color=alt.Color("model"),
        tooltip=[f"count({metric})", "n_sessions", f"mean({metric})"]
    )

    ci = alt.Chart(dataset_df).mark_errorband(opacity=0.5, clip=clip).encode(
        x=alt.X("data_perc_feature_collision"),
        y=alt.Y(metric, scale=alt.Scale(zero=False)),
        color=alt.Color("model", legend=None),
    )

    return alt.layer(
        lines,
        marks,
        ci
    ).resolve_scale(
        color="independent",
        shape="independent"
    )


chart = (
    plot(test_df, legend=True, metric="nDCG@10", title="Synthetic", y=[0.5, 1.0], clip=True)
).configure_legend(
    orient="right",
    title=None,
    labelFont="serif",
    labelFontSize=14,
    columnPadding=20,
).configure_title(
    fontSize=14,
    fontWeight="normal",
    font="serif"
).configure_axis(
    titlePadding=10,
    titleFontSize=14,
    titleFontWeight="normal",
    titleFont="serif",
    labelFontSize=10,
    labelFontWeight="normal",
    labelFont="serif",
    tickCount=8
)

save(chart, "figures/feature_collisions.pdf")
chart